In [1]:
# import mat_properties as prop
# import numpy as n
# import pandas as pd

# RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
# water = prop.Materials_prop(
#     "water",
#     [1.0, 0, 0, 0, 0],
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )
# waterMethane = prop.Materials_prop(
#     "Water*METHANE",
#     (0.833372660622383, 0.166627339377617, 0, 0, 0),
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )
# Methane = prop.Materials_prop(
#     "METHANE",
#     [1.0, 0, 0, 0, 0],
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )
# gas_KU_PKM = prop.Materials_prop(
#     "Nitrogen*Oxygen*CO2*Water*Argon",
#     (0.710320591016015,0.00996710270335893,0.090538556815177,0.180531273012258,0.00864247645319178),
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )

# #Вкл выкл заряд по ПКМ
# PKM_zaryad = True
#####################

gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)


if PKM_zaryad == True:
    
    steamVD_to_turbine = 0.25*water_streams0.at["PEVD-DROSVD", "G"]
    from PKM import steam_transformer
    water_streams.loc["DROSVD-ST", "T":"H"] = water_streams.loc["PEVD-DROSVD", "T":"H"]
    water_streams.loc["DROSVD-ST", "G"] = water_streams.at["PEVD-DROSVD", "G"] - steamVD_to_turbine

    # паротрансформатор
    ST = steam_transformer(stream11="DROSVD-ST",water=water,water_streams=water_streams,heaters=heaters,Pdr1=2,Pdr2=0.8,P2=2,dT=15,dTmin=5,Tdec=10)
    steam_trans = ST.calc()
    
    # Ввод в табл выходов из паротрансформатора
    water_streams.loc["ST-GPK", "T":"G"] = [steam_trans["T17"],steam_trans["P17"],steam_trans["H17"],steam_trans["G1"]]
    water_streams.loc["ST-PKM", "T":"G"] = [steam_trans["T24"],steam_trans["P2"],steam_trans["H24"],steam_trans["G2"]]
    
    
    # реформер
    from PKM import reformer

    ref = reformer(
        stream11="ST-PKM",
        water=water,
        gas_KU=gas_KU_PKM,
        Methane=Methane,
        waterMethane=waterMethane,
        water_streams=water_streams,
        heaters=heaters,
        Tref=700,
        Pref=2,
        T1gas=1968.58395330148,
        T2gas=800,
    )
    reform = ref.calc()

    #Синтезгаз из рефрмера
    syngas_streams.loc["REF-COOL", "T":"G"] = [reform["Tref"],reform["Pref"],reform["Hsg"],reform["Gref"],]
    syngas_streams.loc["REF-COOL", "N2":"CO"] = list(reform["SGfrac"].values())

    # Газы реформера
    gas_streams.loc["AIR-REF", "T":"G"] = [15, 0.1, 414.38, reform["Gair"]]
    gas_streams.loc["CH4-REF", "T":"G"] = [15, 0.7, 881.50, reform["Gch4"]]
    gas_streams.loc["REF-SMESH", "T":"G"] = [800, 0.1, reform["H2gas"], reform["Ggas"]]
    gas_streams.loc["REF-SMESH", "N2":"Ar"] = list(reform["Gasfrac"].values())

    # Смешение
    gas_streams.loc["GTU-PEVD", "G"] = (gas_streams.at["REF-SMESH", "G"] + gas_streams.at["GTU-KU", "G"])
    gas_streams.loc["GTU-PEVD", "H"] = (gas_streams.at["REF-SMESH", "G"] * gas_streams.at["REF-SMESH", "H"]+ gas_streams.at["GTU-KU", "G"] * gas_streams.at["GTU-KU", "H"]) / gas_streams.loc["GTU-PEVD", "G"]
    gas_streams.loc["GTU-PEVD", "P"] = 0.1
    from PKM import mixing_gases_molar
    mixing_gases_molar("GTU-KU", "REF-SMESH", "GTU-PEVD", gas_streams)
    for stream in gas_streams.index[4:10]:
        gas_streams.loc[stream, "N2":"Ar"] = gas_streams.loc["GTU-PEVD", "N2":"Ar"]
        
    #ГПК соединяется в KU_TU.py
    
else:
    gas_streams.loc["GTU-PEVD", "T":"Ar"] = gas_streams.loc["GTU-KU", "T":"Ar"]
    water_streams.loc["ST-GPK", "T":"G"] = [80,2,320,0]
    steamVD_to_turbine=water_streams0.at["PEVD-DROSVD", "G"]
    heaters.loc["Strans":"Ref_HTS", "Qw"] = 0

gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,51.998373,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,568.876864,0.1,1012.407147,555.798373,0.773610,0.112827,0.035878,0.070980,0.006705
PEVD-IVD,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
IVD-EVD,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
EVD-PPND,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
PPND-IND,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
IND-GPK,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
GPK-out,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705


In [2]:
from PKM import PKM_cooler 
cool = PKM_cooler('REF-COOL', 'COOL-HTS', syngas_streams,heaters,450).calc()

print(cool['Qcooler'])

29970.211493591112


In [3]:
from PKM import HTS 
hts = HTS('COOL-HTS', 'HTS-X', syngas_streams,heaters,275).calc()


print(hts['Qhts'])

21696.024996605167


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,45.396191,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,45.396191,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-X,275.0,2.0,2545.703624,45.396191,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029


In [5]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
Strans,130850.501911,NaN,NaN
